In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd 
from tqdm import tqdm
import geopandas as gpd
import mytools

# mrs数据汇总

In [ ]:
#导入atu图层

### 1-导入数据 

In [ ]:
#导入mrs数据到data
f = tools_other.file_name_paths(path='D:/up/mrs/')
data = pd.DataFrame()
tqdm_f=tqdm(f)
for name in tqdm_f:
    data_n = pd.read_csv(name,encoding = 'gbk')
    data_n = data_n[['时间', '小区中文名', 'CGI', '地市', '区县', '厂家名称', '小区类别',
       '自_弱覆盖采样点占比_Eutrancell', '参考信号接收功率采样点点数', '自_自-弱覆盖采样点110_Eutrancell','工作频段']]
    data = data.append(data_n)
    tqdm_f.set_description("进度")

In [ ]:
#按照小区cig进行汇总
data_sum = data.groupby(by = 'CGI').sum()
#去除小区cgi为一列
data_sum.reset_index(inplace = True)

In [ ]:
#导入竞对数据到jingdui
jingdui = pd.read_sql_table('多天指标汇总-mro-19年12月汇总-竞对',con = tools_other.con())

In [ ]:
#导入工参数据到工参
gongcan = pd.read_csv(open('g:/1-规划/工参/工参数据/ltejizhan20191120.csv',encoding = 'gbk'))
gongcan_cig = gongcan.set_index('小区CGI(*)')

In [ ]:
#导入atu图层
atu = gpd.read_file('g:/1-规划/图层/ATU测试区域（201604）/ATU测试区域/ATU网格总.TAB')

#地理查询是atu的小区数
data_t = gpd.sjoin(atu,tools_gis.add_points(gongcan,'RRU经度', 'RRU纬度'))
#新增atu列
data_t = data_t[['小区CGI(*)','所属地区']]
data_t['atu']='是'
data_t = data_t.set_index('小区CGI(*)')

In [ ]:
data_t.drop_duplicates().shape

### 2-数据处理

In [ ]:
#查看数据的完整性
data_sum.shape,gongcan.shape,jingdui.shape

In [ ]:
#数据融合
res =jingdui.merge(data_sum,how = 'outer',left_on = 'cgi',right_on = 'CGI' , indicator = True)
# #查勘比例
# res['tongji'] = res['_merge']
# res.groupby(by = '_merge').agg({'tongji':'count'})
#只保留mrs的小区
res_g = res.loc[res['_merge']!='left_only']
res_g['cgi'] = res_g['CGI']

In [ ]:
#将mrs的覆盖率传递给融合数据
res_g['rsrp_count'] = res_g['参考信号接收功率采样点点数']
res_g['rsrp_weak'] = res_g['自_自-弱覆盖采样点110_Eutrancell']
#只保留和多天指标汇总数据格式一样的列
res_g = res_g[jingdui.columns]

In [ ]:
mrs_jingdui = res_g.set_index('cgi')
#使用gongcan更新数据
mrs_jingdui['rq'] = 201911
mrs_jingdui['atu'] = ''
mrs_jingdui['小区中文名'].update(gongcan_cig['小区中文名'])
mrs_jingdui['city'].update(gongcan_cig['所属地区'])
mrs_jingdui['vander'].update(gongcan_cig['厂家名称'])
mrs_jingdui['gsqx'].update(gongcan_cig['归属区县'])
mrs_jingdui['网络制式(TDD/FDD)'].update(gongcan_cig['网络制式(TDD/FDD)'])
mrs_jingdui['atu'].update(data_t['atu'])

In [ ]:
mro_10_reindex = mrs_jingdui.copy()

#更新竞对黑点数据-数据统一
mro_10_reindex['hd_chun'] = 'f'
mro_10_reindex['hd_chte'] = 'f'
mro_10_reindex['ok_chun'] = 'f'
mro_10_reindex['ok_chte'] = 'f'
#数据格式处理
mro_10_reindex['rsrp_count'] = pd.to_numeric(mro_10_reindex['rsrp_count'])
mro_10_reindex['rsrp_weak'] = pd.to_numeric(mro_10_reindex['rsrp_weak'])
mro_10_reindex['un_maxrsrp_count'] = pd.to_numeric(mro_10_reindex['un_maxrsrp_count'])
mro_10_reindex['un_maxrsrp_weak113'] = pd.to_numeric(mro_10_reindex['un_maxrsrp_weak113'])
mro_10_reindex['te_maxrsrp_count'] = pd.to_numeric(mro_10_reindex['te_maxrsrp_count'])
mro_10_reindex['te_maxrsrp_weak113'] = pd.to_numeric(mro_10_reindex['te_maxrsrp_weak113'])
#覆盖率更新
mro_10_reindex['fgl_cmcc'] = 1 - mro_10_reindex['rsrp_weak'] / mro_10_reindex['rsrp_count']
mro_10_reindex['fgl_chun'] = 1 - mro_10_reindex['un_maxrsrp_weak113'] / mro_10_reindex['un_maxrsrp_count']
mro_10_reindex['fgl_chte'] = 1 - mro_10_reindex['te_maxrsrp_weak113'] / mro_10_reindex['te_maxrsrp_count']

In [ ]:
#竞对更新
mro_10_reindex.loc[((1-mro_10_reindex['rsrp_weak']/mro_10_reindex['rsrp_count']<0.8)
                  &(1-mro_10_reindex['un_maxrsrp_weak113']/mro_10_reindex['un_maxrsrp_count']>0.8))
            |
              ((1-mro_10_reindex['rsrp_weak']/mro_10_reindex['rsrp_count']>0.8)
                  &(((1-mro_10_reindex['un_maxrsrp_weak113']/mro_10_reindex['un_maxrsrp_count'])
                      -(1-mro_10_reindex['rsrp_weak']/mro_10_reindex['rsrp_count']))>0.05))
                   ,'hd_chun'
             ]='t'

mro_10_reindex.loc[((1-mro_10_reindex['rsrp_weak']/mro_10_reindex['rsrp_count']<0.8)
                  &(1-mro_10_reindex['te_maxrsrp_weak113']/mro_10_reindex['te_maxrsrp_count']>0.8))
            |
              ((1-mro_10_reindex['rsrp_weak']/mro_10_reindex['rsrp_count']>0.8)
                  &(((1-mro_10_reindex['te_maxrsrp_weak113']/mro_10_reindex['te_maxrsrp_count'])-
                     (1-mro_10_reindex['rsrp_weak']/mro_10_reindex['rsrp_count']))>0.05))
                   ,'hd_chte'
             ]='t'
#竞对更新
mro_10_reindex.loc[mro_10_reindex['un_maxrsrp_count']>=1000,'ok_chun']='t'
mro_10_reindex.loc[mro_10_reindex['te_maxrsrp_count']>=1000,'ok_chte']='t'
mro_10_reindex_res = mro_10_reindex.reset_index()
mro_10_reindex_res1 = mro_10_reindex_res[jingdui.columns]

### 3-数据导出

In [179]:
#导出到csv
mro_10_reindex_res1.to_csv('G:/1-规划/MRO/1、原始数据/19年12月原始数据/临时-mro12月.csv',encoding = 'gbk',index = False)

NameError: name 'mro_10_reindex_res1' is not defined

In [ ]:
from sqlalchemy.types import Integer,NVARCHAR,Float
def mapping_df_types(df):
    dtypedict = {}
    for i, j in zip(df.columns, df.dtypes):
        if "object" in str(j):
            dtypedict.update({i: NVARCHAR(length=255)})
        if "float" in str(j):
            dtypedict.update({i: Float(precision=2, asdecimal=True)})
        if "int" in str(j):
            dtypedict.update({i: Integer()})
    return dtypedict

In [ ]:
dtypedict = mapping_df_types(mro_10_reindex_res1)
#导出到sql
mro_10_reindex_res1.to_sql(name='多天指标汇总-mro-19年12月汇总-本网替换后', con=tools_other.con(),  index=False, dtype=dtypedict)

### FDD地市覆盖率统计 

In [ ]:
#读取mrs采集数据
data = pd.read_csv('mrs_19_6day.csv',encoding = 'gbk')

In [ ]:
data.columns

In [ ]:
data_fdd = data.loc[data['小区类别']=='TDD']

In [ ]:
data_fdd.groupby(by = '地市').agg({'参考信号接收功率采样点点数':'sum','自_自-弱覆盖采样点110_Eutrancell':'sum'}).to_clipboard()

# 四维度进度

In [3]:
qingdan_yuan1 = pd.read_excel('G:/1-规划/崔开通崔可研规划/2020年任务清单/20200422/副本任务清单-建设类20200422-输出不含李轲.xlsx')

In [4]:
qingdan_yuan1['TDL规划站号（新）'] = qingdan_yuan1['TDL规划站号（新）'].fillna('无')

In [5]:
# qingdan_yuan = qingdan_yuan1.loc[qingdan_yuan1['分类']!='新单子-暂不使用']
qingdan_yuan = qingdan_yuan1.loc[qingdan_yuan1['是否有效数据']=='是']

## 优化统计

In [6]:
qingdan = qingdan_yuan1.copy()

In [7]:
youhua_z = qingdan.loc[(qingdan['优化']=='优化') & (qingdan['优化已派发']==1)]['唯一编号'].count()

In [8]:
youhua_w = qingdan.loc[(qingdan['优化']=='优化') & (qingdan['优化已派发']==1)
                      & (qingdan['优化完成']==1)]['唯一编号'].count()

In [9]:
youhua = qingdan.loc[(qingdan['优化']=='优化') & (qingdan['优化已派发']==1)]

In [10]:
youhuawancheng = qingdan.loc[(qingdan['优化']=='优化') & (qingdan['优化已派发']==1) & (qingdan['优化完成']==1)]

In [11]:
y_w=youhuawancheng.groupby(by = '地市').agg({'唯一编号':'count'})

In [12]:
y_z = youhua.groupby(by = '地市').agg({'唯一编号':'count'})

In [13]:
youhuajindu=pd.concat([y_z,y_w],axis = 1)

In [14]:
youhuajindu.columns = ['总量','完成量']

In [15]:
youhuajindu['进度'] = youhuajindu['完成量']/youhuajindu['总量']

In [16]:
youhua_wanchengdugao=list(youhuajindu.sort_values(by="进度",ascending= False).head(3).index)  #排序 升序True 降序False by列

In [17]:
'MR竞对优化工单目前累计派单'+str(youhua_z)+'单。除328正在处理中，已处理工单覆盖率改善'+str(youhua_w)+'单（本月新增解决181单），派发工单解决\
率'+str(youhua_w/youhua_z*100)[:5]+'%'+'。未解决原因主要是优化无法解决需重点优先开通站点。优化完成工单MR覆盖率改善1.56pp，对全网劣于\
竞对小区占比指标改善3.17pp。'+str(youhua_wanchengdugao[0])+'，'+str(youhua_wanchengdugao[1])+'，'+str(youhua_wanchengdugao[2])+'派发工单解决率相对较高'

'MR竞对优化工单目前累计派单14269单。除328正在处理中，已处理工单覆盖率改善11380单（本月新增解决181单），派发工单解决率79.75%。未解决原因主要是优化无法解决需重点优先开通站点。优化完成工单MR覆盖率改善1.56pp，对全网劣于竞对小区占比指标改善3.17pp。宜昌，恩施，黄冈派发工单解决率相对较高'

In [18]:
youhuajindu.sort_values(by="进度",ascending= False)

,总量,完成量,进度
地市,,,
宜昌,1391,1208,0.868440
恩施,1407,1205,0.856432
黄冈,1381,1164,0.842867
孝感,866,724,0.836028
鄂州,396,330,0.833333
潜江,226,183,0.809735
江汉,342,273,0.798246
黄石,527,418,0.793169
随州,648,508,0.783951


## 催开进度

In [19]:
qingdan.columns

Index(['唯一编号', '问题点或问题小区cgi', '对应cgi', '所属网洞ID_', '问题点中心经度', '问题点中心纬度',
       '是否计划内', '分类', '地市', '区县', '厂家', '是否黑点', '场景_农村', '场景_通报场景', '场景_室内外',
       '美食', '景区', '乡镇', '核心城区', '场景', '优化', '规划', '催可研', '催开', '方案-最新',
       'TDL规划站号（新）', '规划站名', '逻辑站号', '站型', '工期核实', '优化已派发', '优化派发时间', '派发处方案',
       '优化完成', '规划完成', '可研完成', '催开完成', '备注', '是否有效数据'],
      dtype='object')

In [20]:
qingdan = qingdan_yuan.copy()

In [21]:
cuikai_zdanzi = qingdan.loc[(qingdan['方案-最新']=='催开') ]['唯一编号'].count()

In [22]:
cuikai_z = qingdan.loc[(qingdan['方案-最新']=='催开') ].drop_duplicates(subset='TDL规划站号（新）') ['唯一编号'].count()

In [23]:
cuikai_w = qingdan.loc[(qingdan['方案-最新']=='催开') & (qingdan['催开完成']==1) ].drop_duplicates(subset='TDL规划站号（新）') ['唯一编号'].count()

In [24]:
cuikai_quchong = qingdan.loc[(qingdan['方案-最新']=='催开') ].drop_duplicates(subset='TDL规划站号（新）') 

In [25]:
cuikai_quchong_wancheng = qingdan.loc[(qingdan['方案-最新']=='催开') & (qingdan['催开完成']==1) ].drop_duplicates(subset='TDL规划站号（新）') 

In [26]:
cuikai_zongliang = cuikai_quchong.groupby(by = '地市').agg({'唯一编号':'count'})

In [27]:
cuikai_wanchengliang = cuikai_quchong_wancheng.groupby(by = '地市').agg({'唯一编号':'count'})

In [28]:
cuikaijindu=pd.concat([cuikai_zongliang,cuikai_wanchengliang],axis = 1)

In [29]:
cuikaijindu.columns = ['总量','完成量']

In [30]:
cuikaijindu['进度'] = cuikaijindu['完成量']/cuikaijindu['总量']

In [31]:
cuikai_wanchengdudi=list(cuikaijindu.sort_values(by="进度",ascending= True).head(1).index)  #排序 升序True 降序False by列

In [32]:
a=cuikai_wanchengdudi[0]

In [33]:
jindudi=cuikai_quchong.loc[(cuikai_quchong['地市']==a) & (cuikai_quchong['催开完成']!=1)].groupby(by='工期核实').agg({'唯一编号':'count'}).sort_values(by="唯一编号",ascending= False).head(1).index

In [34]:
'催开工单共'+str(cuikai_zdanzi)+'单，涉及逻辑站\
点'+str(cuikai_z)+'个，已开通站点'+str(cuikai_w)+'个(本月新增201个），催开完成率\
为'+str(cuikai_w/cuikai_z*100)[:5]+'%'+'。'+cuikai_wanchengdudi[0]+'地市整体解决进度较慢，主要为'+jindudi[0]+'未开通占比较\
高，需加快站点建设进度。'+'已催开工单整体MR覆盖率改善2.60pp，对全网劣于竞对小区占比指标改善5.26pp。'

'催开工单共26820单，涉及逻辑站点15018个，已开通站点13303个(本月新增201个），催开完成率为88.58%。十堰地市整体解决进度较慢，主要为农村4G覆盖一期工程第一批未开通占比较高，需加快站点建设进度。已催开工单整体MR覆盖率改善2.60pp，对全网劣于竞对小区占比指标改善5.26pp。'

In [35]:
cuikaijindu.sort_values(by="进度",ascending= True)

,总量,完成量,进度
地市,,,
十堰,694,480,0.691643
武汉,4810,3618,0.752183
鄂州,150,134,0.893333
天门,208,186,0.894231
荆州,403,369,0.915633
黄石,408,380,0.931373
襄阳,901,854,0.947836
孝感,719,693,0.963839
潜江,209,203,0.971292


## 可研统计

In [36]:
qingdan = qingdan_yuan.copy()

In [37]:
keyan_zdanzi = qingdan.loc[(qingdan['催可研']=='可研') ]['唯一编号'].count()

In [38]:
keyan_z = qingdan.loc[(qingdan['催可研']=='可研') ].drop_duplicates(subset='TDL规划站号（新）') ['唯一编号'].count()

In [39]:
keyan_w = qingdan.loc[(qingdan['催可研']=='可研') & (qingdan['可研完成']==1) ].drop_duplicates(subset='TDL规划站号（新）') ['唯一编号'].count()

In [40]:
'催可研工单共'+str(keyan_zdanzi)+'单，涉及'+str(keyan_z)+'个逻辑站点工期催研，目前累\
计'+str(keyan_w)+'个站点已纳入工期，入研进度为'+str(keyan_w/keyan_z*100)[:5]+'%。'

'催可研工单共26423单，涉及14878个逻辑站点工期催研，目前累计14793个站点已纳入工期，入研进度为99.42%。'

## 规划进度

In [41]:
qingdan = qingdan_yuan.copy()

In [42]:
guihua_zdanzi = qingdan.loc[(qingdan['规划']=='规划') ]['唯一编号'].count()

In [43]:
guihua_w = qingdan.loc[(qingdan['规划']=='规划') & (qingdan['规划完成']==1) ]['唯一编号'].count()

In [44]:
'需规划解决工单共'+str(guihua_zdanzi)+'单，目前累计完成'+str(guihua_w)+'个问题工单规划衔接，规\
划完成比例'+str(guihua_w/guihua_zdanzi*100)[:5]+'%。'

'需规划解决工单共6081单，目前累计完成6070个问题工单规划衔接，规划完成比例99.81%。'

# ------武汉竞对及MR分析

### 优化

In [45]:
qingdan_y = qingdan_yuan1.copy()

In [46]:
qingdan_y = qingdan_y.loc[qingdan_y['地市']=='武汉']

In [47]:
qingdan = qingdan_y.copy()

In [48]:
youhua_z = qingdan.loc[(qingdan['优化']=='优化') & (qingdan['优化已派发']==1)]['唯一编号'].count()

In [49]:
youhua_w = qingdan.loc[(qingdan['优化']=='优化') & (qingdan['优化已派发']==1)
                      & (qingdan['优化完成']==1)]['唯一编号'].count()

In [50]:
youhua = qingdan.loc[(qingdan['优化']=='优化') & (qingdan['优化已派发']==1)]

In [51]:
youhuawancheng = qingdan.loc[(qingdan['优化']=='优化') & (qingdan['优化已派发']==1) & (qingdan['优化完成']==1)]

In [52]:
y_w=youhuawancheng.groupby(by = '区县').agg({'唯一编号':'count'})

In [53]:
y_z = youhua.groupby(by = '区县').agg({'唯一编号':'count'})

In [54]:
youhuajindu=pd.concat([y_z,y_w],axis = 1)

In [55]:
youhuajindu.columns = ['总量','完成量']

In [56]:
youhuajindu['进度'] = youhuajindu['完成量']/youhuajindu['总量']

In [57]:
youhua_wanchengdugao=list(youhuajindu.sort_values(by="进度",ascending= False).head(3).index)  #排序 升序True 降序False by列

In [58]:
'MR竞对优化工单目前累计派单'+str(youhua_z)+'单。除328正在处理中，已处理工单覆盖率改善'+str(youhua_w)+'单（本月新增解决181单），派发工单解决\
率'+str(youhua_w/youhua_z*100)[:5]+'%'+'。未解决原因主要是优化无法解决需重点优先开通站点。优化完成工单MR覆盖率改善1.56pp，对全网劣于\
竞对小区占比指标改善3.17pp。'+str(youhua_wanchengdugao[0])+'，'+str(youhua_wanchengdugao[1])+'，'+str(youhua_wanchengdugao[2])+'派发工单解决率相对较高'

'MR竞对优化工单目前累计派单3541单。除328正在处理中，已处理工单覆盖率改善2704单（本月新增解决181单），派发工单解决率76.36%。未解决原因主要是优化无法解决需重点优先开通站点。优化完成工单MR覆盖率改善1.56pp，对全网劣于竞对小区占比指标改善3.17pp。汉阳区，江汉区，洪山区派发工单解决率相对较高'

In [127]:
youhuajindu.to_clipboard()

### 催开

In [79]:
qingdan_c = qingdan_yuan.copy()

In [80]:
qingdan_c = qingdan_c.loc[qingdan_c['地市']=='武汉']

In [81]:
qingdan = qingdan_c.copy()

In [82]:
cuikai_zdanzi = qingdan.loc[(qingdan['方案-最新']=='催开') ]['唯一编号'].count()

In [83]:
cuikai_z = qingdan.loc[(qingdan['方案-最新']=='催开') ].drop_duplicates(subset='TDL规划站号（新）') ['唯一编号'].count()

In [84]:
cuikai_w = qingdan.loc[(qingdan['方案-最新']=='催开') & (qingdan['催开完成']==1) ].drop_duplicates(subset='TDL规划站号（新）') ['唯一编号'].count()

In [85]:
cuikai_quchong = qingdan.loc[(qingdan['方案-最新']=='催开') ].drop_duplicates(subset='TDL规划站号（新）') 

In [86]:
cuikai_quchong_wancheng = qingdan.loc[(qingdan['方案-最新']=='催开') & (qingdan['催开完成']==1) ].drop_duplicates(subset='TDL规划站号（新）') 

In [87]:
cuikai_zongliang = cuikai_quchong.groupby(by = '区县').agg({'唯一编号':'count'})

In [88]:
cuikai_wanchengliang = cuikai_quchong_wancheng.groupby(by = '区县').agg({'唯一编号':'count'})

In [89]:
cuikaijindu=pd.concat([cuikai_zongliang,cuikai_wanchengliang],axis = 1)

In [90]:
cuikaijindu.columns = ['总量','完成量']

In [91]:
cuikaijindu['进度'] = cuikaijindu['完成量']/cuikaijindu['总量']

In [92]:
cuikai_wanchengdudi=list(cuikaijindu.sort_values(by="进度",ascending= True).head(1).index)  #排序 升序True 降序False by列

In [93]:
a=cuikai_wanchengdudi[0]

In [94]:
jindudi=cuikai_quchong.loc[(cuikai_quchong['区县']==a) & (cuikai_quchong['催开完成']!=1)].groupby(by='工期核实').agg({'唯一编号':'count'}).sort_values(by="唯一编号",ascending= False).head(1).index

In [95]:
'催开工单共'+str(cuikai_zdanzi)+'单，涉及逻辑站\
点'+str(cuikai_z)+'个，已开通站点'+str(cuikai_w)+'个(本月新增201个），催开完成率\
为'+str(cuikai_w/cuikai_z*100)[:5]+'%'+'。'+cuikai_wanchengdudi[0]+'地市整体解决进度较慢，主要为'+jindudi[0]+'未开通占比较\
高，需加快站点建设进度。'+'已催开工单整体MR覆盖率改善2.60pp，对全网劣于竞对小区占比指标改善5.26pp。'

'催开工单共10371单，涉及逻辑站点4818个，已开通站点3626个(本月新增201个），催开完成率为75.25%。东湖高新开发区地市整体解决进度较慢，主要为4G六期第二批未开通占比较高，需加快站点建设进度。已催开工单整体MR覆盖率改善2.60pp，对全网劣于竞对小区占比指标改善5.26pp。'

In [96]:
cuikaijindu

,总量,完成量,进度
区县,,,
东湖高新开发区,705,406,0.575887
东西湖区,338,285,0.843195
新洲县,288,267,0.927083
武昌区,319,201,0.630094
武汉市,57,50,0.877193
汉阳区,221,160,0.723982
江夏区,430,340,0.790698
江岸区,264,198,0.750000
江汉区,226,138,0.610619


In [128]:
cuikaijindu.to_clipboard()

### 可研

In [98]:
qingdan_k = qingdan_yuan.copy()

In [99]:
qingdan_k = qingdan_k.loc[qingdan_k['地市']=='武汉']

In [100]:
qingdan = qingdan_k.copy()

In [101]:
keyan_zdanzi = qingdan.loc[(qingdan['催可研']=='可研') ]['唯一编号'].count()

In [102]:
keyan_z = qingdan.loc[(qingdan['催可研']=='可研') ].drop_duplicates(subset='TDL规划站号（新）') ['唯一编号'].count()

In [103]:
keyan_w = qingdan.loc[(qingdan['催可研']=='可研') & (qingdan['可研完成']==1) ].drop_duplicates(subset='TDL规划站号（新）') ['唯一编号'].count()

In [104]:
'催可研工单共'+str(keyan_zdanzi)+'单，涉及'+str(keyan_z)+'个逻辑站点工期催研，目前累\
计'+str(keyan_w)+'个站点已纳入工期，入研进度为'+str(keyan_w/keyan_z*100)[:5]+'%。'

'催可研工单共9891单，涉及4623个逻辑站点工期催研，目前累计4611个站点已纳入工期，入研进度为99.74%。'

In [105]:
keyan_quchong = qingdan.loc[(qingdan['催可研']=='可研') ].drop_duplicates(subset='TDL规划站号（新）')

In [106]:
keyan_quchong_w = qingdan.loc[(qingdan['催可研']=='可研') & (qingdan['可研完成']==1) ].drop_duplicates(subset='TDL规划站号（新）')

In [107]:
k_z = keyan_quchong.groupby(by = '区县').agg({'唯一编号':'count'})

In [108]:
k_w = keyan_quchong_w.groupby(by = '区县').agg({'唯一编号':'count'})

In [109]:
keyanjindu=pd.concat([k_z,k_w],axis = 1)

In [110]:
keyanjindu.columns = ['总量','完成量']

In [111]:
keyanjindu['进度'] = keyanjindu['完成量']/keyanjindu['总量']

In [129]:
keyanjindu.to_clipboard()

### 规划 

In [113]:
qingdan_g = qingdan_yuan.copy()

In [114]:
qingdan_g = qingdan_g.loc[qingdan_g['地市']=='武汉']

In [115]:
qingdan = qingdan_g.copy()

In [116]:
guihua_zdanzi = qingdan.loc[(qingdan['规划']=='规划') ]['唯一编号'].count()

In [117]:
guihua_w = qingdan.loc[(qingdan['规划']=='规划') & (qingdan['规划完成']==1) ]['唯一编号'].count()

In [118]:
'需规划解决工单共'+str(guihua_zdanzi)+'单，目前累计完成'+str(guihua_w)+'个问题工单规划衔接，规\
划完成比例'+str(guihua_w/guihua_zdanzi*100)[:5]+'%。'

'需规划解决工单共2124单，目前累计完成2124个问题工单规划衔接，规划完成比例100.0%。'

In [119]:
guihua_qingdan = qingdan.loc[(qingdan['规划']=='规划') ]

In [120]:
guihua_qingdan_w = qingdan.loc[(qingdan['规划']=='规划') & (qingdan['规划完成']==1)]

In [121]:
g_z = guihua_qingdan.groupby(by = '区县').agg({'唯一编号':'count'})

In [122]:
g_w = guihua_qingdan_w.groupby(by = '区县').agg({'唯一编号':'count'})

In [123]:
guihuajindu=pd.concat([g_z,g_w],axis = 1)

In [124]:
guihuajindu.columns = ['总量','完成量']

In [125]:
guihuajindu['进度'] = guihuajindu['完成量']/guihuajindu['总量']

In [130]:
guihuajindu.to_clipboard()